In [ ]:
# conda install -c conda-forge python-annoy

In [1]:
import os
import string
import annoy

from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
from gensim.models import Word2Vec

import numpy as np
from tqdm import tqdm_notebook
import pandas as pd

ModuleNotFoundError: No module named 'gensim'

Предобработаем ответы mail.ru из файла: к каждому вопросу присоединим 1 ответ и запишем в файл на будущее. Это позволит нам сэкономить время и ресурсы при дальнейшем препроцессинге текста

In [ ]:
question = None
written = False

#Мы идем по всем записям, берем первую строку как вопрос
# и после знака --- находим ответ
with open("prepared_answers.txt", "w") as fout:
    with open("Otvety.txt", "r") as fin:
        for line in tqdm_notebook(fin):
            if line.startswith("---"):
                written = False
                continue
            if not written and question is not None:
                fout.write(question.replace("\t", " ").strip() + "\t" + line.replace("\t", " "))
                written = True
                question = None
                continue
            if not written:
                question = line.strip()
                continue

Теперь нам нужно предобработать текст, чтобы обучить word2vec и получить эмбеддинги. Удаляем знаки препинания и делаем лемматизацию

In [ ]:
def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [ ]:
sentences = []

morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)
c = 0

with open("Otvety.txt", "r") as fin:
    for line in tqdm_notebook(fin):
        spls = preprocess_txt(line)
        sentences.append(spls)
        c += 1
        if c > 500000:
            break

In [ ]:
# Обучим модель word2vec на наших вопросах
sentences = [i for i in sentences if len(i) > 2]
model = Word2Vec(sentences=sentences, vector_size=100, min_count=1, window=5)
model.save("w2v_model")

Теперь нам нужно сложить в индекс все вопросы. Используем библиотеку annoy. Проходимся по всем ответам, считаем, что вектор предложения - сумма word2vecов слов, которые входят в него (конечно же усредненная)

In [ ]:
index = annoy.AnnoyIndex(100 ,'angular')

index_map = {}
counter = 0

with open("prepared_answers.txt", "r") as f:
    for line in tqdm_notebook(f):
        n_w2v = 0
        spls = line.split("\t")
        index_map[counter] = spls[1]
        question = preprocess_txt(spls[0])
        vector = np.zeros(100)
        for word in question:
            if word in model.wv:
                vector += model.wv[word]
                n_w2v += 1
        if n_w2v > 0:
            vector = vector / n_w2v
        index.add_item(counter, vector)
            
        counter += 1

index.build(10)
index.save('speaker.ann')

Теперь остается реализовать метод, который получит на вход вопрос и найдет ответ к нему!
Мы препроцессим вопрос, находим ближайший вопрос и выбираем ответ на ближайший вопрос.

In [ ]:
def find_answer(question):
    preprocessed_question = preprocess_txt(question)
    n_w2v = 0
    vector = np.zeros(100)
    for word in preprocessed_question:
        if word in model.wv:
            vector += model.wv[word]
            n_w2v += 1
    if n_w2v > 0:
        vector = vector / n_w2v
    answer_index = index.get_nns_by_vector(vector, 1)
    return index_map[answer_index[0]]